# Visualization of results and data

In this notebook we visualize some of the the data and results from our experiments.

In [1]:
import os

import mlflow

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import ipywidgets as widgets
from ipywidgets import Layout, Box
%matplotlib widget

In [2]:
# setup for src emotion images
emots = ['happy', 'angry', 'sad', 'surprise']

# setup for generated images
df = mlflow.search_runs([str(0)])
models = {k: v for k, v in  df[~(df['tags.spectrogram'].isnull())][['tags.mlflow.runName', 'run_id']].values}


In [25]:
# Helpers
def get_emot_img(emot):
    tst_img_dir = os.path.join('data','FER', 'test', emot)
    _file = os.listdir(tst_img_dir)[0]
    img_pth = os.path.join(tst_img_dir, _file)
    return mpimg.imread(img_pth), _file

def get_spect_at_epoch(run_id, epoch, emote, sample_name):
    if epoch == 0:
        epoch = 1
    model_pth  = os.path.join('mlruns', '0', run_id, 'artifacts', str(epoch), emote + '-' + sample_name+'.npy')
    return np.load(model_pth)

def display_results(obj):
    img, img_name = get_emot_img(emot_widget.value)
    spect = get_spect_at_epoch(models[mdl_widget.value], epoch_widget.value, emot_widget.value, img_name)
    fig, (ax1, ax2) = plt.subplots(1, 2)
    ax1.imshow(img, cmap='gray')
    ax2.imshow(spect[0][0], cmap='gray')
    output.clear_output()
    with output:
        fig.show()

In [26]:
# widget setup

emot_widget = widgets.Dropdown(
    options=emots,
    value=emots[0],
    description='Emotions:',
    disabled=False,
)

mdl_widget = widgets.Dropdown(
    options=list(models.keys()),
    value=list(models.keys())[0],
    description='Models:',
    disabled=False,
)

epoch_widget = widgets.IntSlider(
    value=300,
    min=0,
    max=300,
    step=15,
    description='Epoch:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

items_layout = Layout(height='auto') 

button = widgets.Button(description="Click Me!")
output = widgets.Output(layout=items_layout)

box_layout = Layout(display='flex',
                    flex_flow='column',
                    align_items='stretch')

items = [emot_widget, mdl_widget, epoch_widget, button, output]
box = Box(children=items, layout=box_layout)
button.on_click(display_results)

In [27]:
box


Box(children=(Dropdown(description='Emotions:', options=('happy', 'angry', 'sad', 'surprise'), value='happy'),…